In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

connection_string = os.getenv("VECTOR_DATABASE_URL")

if not connection_string:
    raise ValueError("VECTOR_DATABASE_URL tidak ditemukan di file .env")

API_KEY = os.getenv("OPENROUTER_API_KEY")
BASE_URL = os.getenv("OPENROUTER_BASE_URL")

print("Environment variables loaded.")

Environment variables loaded.


In [2]:
import json
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

json_files = ["../data/skj/skj_kepala_seksi.json", "../data/skj/skj_inspektur.json"]

def load_skj_to_documents(file_paths):
    documents = []
    
    for file_path in file_paths:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
            jabatan = data.get('jabatan', 'Unknown Position')
            source = data.get('source_file', file_path)
            
            for komp in data.get('kompetensi', []):
                nama_komp = komp.get('nama_kompetensi', '')
                definisi = komp.get('definisi', '')
                
                indikator = ", ".join(komp.get('indikator_perilaku', []))
                
                level_desc = komp.get('level_mapping', {})
                
                page_content = (
                    f"Jabatan: {jabatan}\n"
                    f"Kompetensi: {nama_komp}\n"
                    f"Definisi: {definisi}\n"
                    f"Indikator Perilaku: {indikator}\n"
                    f"Level Proficiency: {json.dumps(level_desc, indent=0)}" 
                )
                
                metadata = {
                    "source": source,
                    "jabatan": jabatan,
                    "kompetensi": nama_komp
                }
                
                documents.append(Document(page_content=page_content, metadata=metadata))
                
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
            
    return documents

print("Memuat file JSON...")
docs = load_skj_to_documents(json_files)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

splits = text_splitter.split_documents(docs)

print(f"Berhasil! Dokumen diproses menjadi {len(splits)} potongan (chunks).")
print(f"Contoh chunk pertama:\n{splits[0].page_content[:200]}...")

d:\Zulfi\My-Skripsi-Gweh\KP\demo-penilaian-kualitatif\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Memuat file JSON...
Berhasil! Dokumen diproses menjadi 4 potongan (chunks).
Contoh chunk pertama:
Jabatan: Kepala Seksi
Kompetensi: Pengambilan Keputusan
Definisi: Kemampuan membuat keputusan yang tepat berbasis data, risiko, dan tujuan unit.
Indikator Perilaku: Mengidentifikasi informasi penting ...


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_postgres import PGVector
from langchain_openai import OpenAIEmbeddings


print("Memuat model embedding...")
# embeddings = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-MiniLM-L6-v2",
#     model_kwargs={'device': 'cpu'},
#     encode_kwargs={'normalize_embeddings': False}
# )

embeddings = OpenAIEmbeddings(
    base_url=BASE_URL,
    api_key=API_KEY, 
    model="qwen/qwen3-embedding-8b"   
) 

COLLECTION_NAME = "skj_index" 

vectorstore = PGVector(
    embeddings=embeddings,
    collection_name=COLLECTION_NAME,
    connection=connection_string,
    use_jsonb=True,
)

print(f"Menyimpan vector ke tabel '{COLLECTION_NAME}' di Postgres...")
vectorstore.add_documents(splits)
print("Penyimpanan selesai!")

Memuat model embedding...
Menyimpan vector ke tabel 'skj_index' di Postgres...
Penyimpanan selesai!


In [4]:
from langchain_openai import ChatOpenAI
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model="x-ai/grok-4.1-fast:free", 
    api_key=API_KEY,
    base_url=BASE_URL,
    temperature=0
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

system_prompt = (
    "Anda adalah asisten AI yang membantu menjawab pertanyaan berdasarkan konteks yang diberikan di bawah ini. "
    "Jika jawaban tidak ada di dalam konteks, katakan 'Saya tidak menemukan informasi tersebut di dokumen'. "
    "Jawablah dengan bahasa Indonesia yang jelas.\n\n"
    "Konteks:\n{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("RAG Chain siap digunakan!")

RAG Chain siap digunakan!


In [5]:
query = "apa inti utama dokumen ini?"

print(f"Pertanyaan: {query}")
print("-" * 30)

response = rag_chain.invoke({"input": query})

print("JAWABAN AI:")
print(response["answer"])

print("\n" + "-" * 30)
print("Sumber Referensi:")
for i, doc in enumerate(response["context"]):
    print(f"[{i+1}] Halaman {doc.metadata.get('page', '?')}: {doc.page_content[:1000]}...")

Pertanyaan: apa inti utama dokumen ini?
------------------------------
JAWABAN AI:
Inti utama dokumen ini adalah deskripsi kompetensi jabatan, yang mencakup definisi, indikator perilaku, dan level proficiency (1-5) untuk:

- **Jabatan Kepala Seksi**: Kompetensi Kepemimpinan dan Pengambilan Keputusan.
- **Jabatan Administrator**: Kompetensi Kerjasama.

------------------------------
Sumber Referensi:
[1] Halaman ?: Jabatan: Kepala Seksi
Kompetensi: Kepemimpinan
Definisi: Kemampuan memimpin, mengarahkan, dan memotivasi tim untuk mencapai tujuan.
Indikator Perilaku: Memberikan arahan kerja yang jelas., Memotivasi anggota tim., Menjadi role model dalam sikap kerja.
Level Proficiency: {
"1": "Memimpin tugas sederhana.",
"2": "Mengarahkan pekerjaan tim kecil.",
"3": "Memimpin tim secara efektif dalam berbagai situasi.",
"4": "Mengelola dinamika tim tingkat unit.",
"5": "Menjadi pemimpin strategis organisasi."
}...
[2] Halaman ?: Jabatan: Kepala Seksi
Kompetensi: Pengambilan Keputusan
Definis